# HWDP

$ x = \frac{\sqrt{z}}{y} $

In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
db= pd.read_csv(r'filtered_data.csv')

In [3]:
db.head()

,Unnamed: 0,Event,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,Game
0,0,Rated Bullet game,0-1,1079,1071,-30,10,B00,Owen Defense,60+0,Time forfeit,1. e4 { [%eval 0.12] [%clk 0:01:00] } 1... b6?...
1,1,Rated Blitz game,0-1,1888,1796,-13,13,C62,Ruy Lopez: Steinitz Defense,300+0,Normal,1. e4 { [%eval 0.12] [%clk 0:05:00] } 1... e5 ...
2,2,Rated Rapid game,0-1,1090,1156,-9,11,B20,Sicilian Defense,600+0,Normal,1. e4 { [%eval 0.12] [%clk 0:10:00] } 1... c5 ...
3,3,Rated Bullet game,0-1,1091,1184,-8,9,C20,"King's Pawn Game: Wayward Queen Attack, Kiddie...",60+0,Time forfeit,1. e4 { [%eval 0.12] [%clk 0:01:00] } 1... e5 ...
4,4,Rated Classical game,1-0,1672,1699,11,-11,C64,"Ruy Lopez: Classical Variation, Central Variation",900+15,Normal,1. e4 { [%eval 0.12] [%clk 0:15:00] } 1... e5 ...


In [4]:
db.describe()

,Unnamed: 0,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff
count,31461.000000,31461.000000,31461.000000,31461.000000,31461.000000
mean,15730.000000,1503.446903,1503.921967,4.022536,2.654684
std,9082.152746,321.223393,320.695908,30.933538,29.354305
min,0.000000,800.000000,800.000000,-437.000000,-615.000000
25%,7865.000000,1266.000000,1267.000000,-10.000000,-10.000000
50%,15730.000000,1500.000000,1500.000000,3.000000,-2.000000
75%,23595.000000,1725.000000,1724.000000,11.000000,11.000000
max,31460.000000,2748.000000,2667.000000,460.000000,598.000000


In [24]:
# domyślnie bez [:5]
a = db.Game.loc[:5]
a[0]

'1. e4 { [%eval 0.12] [%clk 0:01:00] } 1... b6?! { [%eval 0.62] [%clk 0:01:00] } 2. d4 { [%eval 0.64] [%clk 0:00:57] } 2... Bb7 { [%eval 0.62] [%clk 0:01:00] } 3. Bd3 { [%eval 0.47] [%clk 0:00:57] } 3... g6 { [%eval 0.54] [%clk 0:01:00] } 4. c3 { [%eval 0.27] [%clk 0:00:55] } 4... Bg7 { [%eval 0.68] [%clk 0:00:57] } 5. h3 { [%eval 0.62] [%clk 0:00:51] } 5... e6 { [%eval 0.8] [%clk 0:00:55] } 6. f4?! { [%eval -0.15] [%clk 0:00:48] } 6... Ne7 { [%eval 0.27] [%clk 0:00:54] } 7. Nf3 { [%eval 0.46] [%clk 0:00:46] } 7... d5 { [%eval 0.58] [%clk 0:00:53] } 8. O-O?? { [%eval -5.28] [%clk 0:00:45] } 8... dxe4 { [%eval -5.27] [%clk 0:00:52] } 9. Bxe4 { [%eval -5.76] [%clk 0:00:41] } 9... Bxe4 { [%eval -5.78] [%clk 0:00:51] } 10. Ng5?! { [%eval -6.37] [%clk 0:00:40] } 10... Bb7 { [%eval -6.42] [%clk 0:00:47] } 11. Nxf7? { [%eval -8.64] [%clk 0:00:38] } 11... Kxf7 { [%eval -8.5] [%clk 0:00:45] } 12. Qg4?! { [%eval -9.21] [%clk 0:00:33] } 12... Qd6 { [%eval -8.83] [%clk 0:00:42] } 13. Nd2 { [%eval 

In [25]:
def time_to_sec(hhmmss):
    return sum(x * int(t) for x, t in zip([3600, 60, 1], hhmmss.split(":")))

def derivative(value_list, step=1, reversed=False):
    v_copy = [it for it in value_list]
    for i in range(len(v_copy)-1, -1, -1):
        if i in np.arange(step):
            v_copy[i] = 0
        if i >= step:
            v_copy[i] = v_copy[i] - v_copy[i-step]
    if reversed:
        v_copy = [-1 * it for it in v_copy]
    return v_copy

# na razie na a[0] -> zrobić fora dla wszystkich
ev = re.findall(r"\%eval (.*?)\]",a[0])
ev = [float(it) for it in ev]

t_left = re.findall(r"\%clk (.*?)\]",a[0])
t_left = [time_to_sec(it) for it in t_left]

d_t = derivative(t_left, step=2, reversed=True)
d_ev = [round(it,2) for it in derivative(ev, step=1, reversed=False)]
d_ev

[0,
 0.5,
 0.02,
 -0.02,
 -0.15,
 0.07,
 -0.27,
 0.41,
 -0.06,
 0.18,
 -0.95,
 0.42,
 0.19,
 0.12,
 -5.86,
 0.01,
 -0.49,
 -0.02,
 -0.59,
 -0.05,
 -2.22,
 0.14,
 -0.71,
 0.38,
 -0.01,
 0.45,
 -0.23,
 0.17,
 -0.32,
 0.02,
 -0.05,
 0.07,
 -5.46,
 5.32,
 0.03,
 0.72,
 -0.87,
 0.69,
 0.07,
 0.91,
 0.02,
 0.74,
 -1.31,
 0.06]

In [57]:
x = range(3,-1,-1)
[it for it in x]


[3, 2, 1, 0]